In [2]:
from cryptolib import diffie_hellman_decrypt_flag, json_send
diffie_hellman_decrypt_flag

<function cryptolib.diffie_hellman_decrypt_flag(shared_secret: int, iv: str, ciphertext: str)>

In [20]:
from pwnlib.tubes.remote import remote
from json import loads
r = remote("socket.cryptohack.org", 13379)
payload = loads(r.recvline().decode().split("Intercepted from Alice: ")[1])
supported = payload.values()
payload['supported'] = ["DH64"]
json_send(r, payload)
payload = loads(r.recvline().decode().split("Send to Bob: Intercepted from Bob: ")[1])
json_send(r, payload)
payload = loads(r.recvline().decode().split("Send to Alice: Intercepted from Alice: ")[1])
p, g, A = payload.values()
payload['A'] = "0x01"
payload['B'] = "0x01"
json_send(r, payload)
B = loads(r.recvline().decode().split("Intercepted from Bob: ")[1])['B']
json_send(r, payload)
iv, flag_enc = loads(r.recvline().decode().split("Intercepted from Alice: ")[1]).values()

from sympy import discrete_log
a = discrete_log(int(p, 16), int(A, 16), int(g, 16))
b = discrete_log(int(p, 16), int(B, 16), int(g, 16))
secret = pow(pow(int(g, 16), a, int(p, 16)), b, int(p, 16))
diffie_hellman_decrypt_flag(secret, iv, flag_enc)

b'crypto{d0wn6r4d35_4r3_d4n63r0u5}'